# Test 1 Sam Maske: Achtung nicht löschen !!!!!!!!!!


In [ ]:
from PIL import Image, ImageOps
import torch
import numpy as np

In [ ]:
im_size =[720,720]
mask_path = 'dataset/mini_testdatensatz/segmentation_images/picture_1.png'
img_path = 'dataset/mini_testdatensatz/train/picture_1.png'

In [ ]:
def img_patch(img,segmentation=False):
        # Calculate the aspect ratio of the image
        img_ratio = img.width / img.height

        # If the image is wider than it is high
        if img_ratio > 1:
            # Scale the width to the desired size and calculate the height while maintaining the aspect ratio
            new_width = im_size[0]
            new_height = int(new_width / img_ratio)
        # If the image is taller than it is wide
        else:
            # Scale the height to the desired size and calculate the width while maintaining the aspect ratio
            new_height = im_size[1]
            new_width = int(new_height * img_ratio)

        # Resize the image to the calculated size while maintaining the aspect ratio
        img = img.resize((new_width, new_height), resample=Image.BICUBIC)

        # Add black borders to the left and right or top and bottom of the image to make it the desired size
        delta_w = im_size[0] - new_width
        delta_h = im_size[1] - new_height
        pad_width = delta_w // 2
        pad_height = delta_h // 2
        padding = (pad_width, pad_height, delta_w - pad_width, delta_h - pad_height)
        if segmentation:
            img = ImageOps.expand(img, border=padding, fill=(0))
        else:
            img = ImageOps.expand(img, border=padding, fill=(255,255,255))

        # Return the scaled and centered image
        return img

In [ ]:
# Achtung, mask und mask tensor sind nur auf Grund der Visualisierung von mask aufkommentiert, im echten Code nicht
def img_segmention2(mask_path):

    mask = Image.open(mask_path).convert('1')
    mask = img_patch(mask,segmentation=True) # problem with the color --> we only need one channel
    mask = np.array(mask)
    mask_tensor = torch.from_numpy(mask)

    return mask_tensor, mask

In [ ]:
img = Image.open(img_path)
img = img_patch(img)
img = np.array(img)
display(Image.fromarray(img))


In [ ]:
_,mask = img_segmention2(mask_path)
display(Image.fromarray(mask))

In [ ]:
result_array = np.zeros_like(img)
result_array[mask] = img[mask]
result_image = Image.fromarray(result_array)
display(result_image)

# Test 2 Datensatz analysieren 1c

In [ ]:
import numpy as np

# Create a 3D numpy array with shape (height, width, channels) representing an RGB image
img = np.zeros((50, 50, 3)) # Set all values to 0 initially
img[10:20, 20:30, :] = [0, 0, 255] # Set a region of the image to red

# Create a 2D numpy array with shape (height, width) representing a binary mask
mask = np.zeros((50, 50)) # Set all values to 0 initially
mask[10:25, 20:30] = 1 # Set a rectangular region of the mask to 1

# Check that the shapes of the arrays are the same
#assert img.shape[:-1] == mask.shape


# Calculate the mean of the img array for every channel, only using the values of pixels where the corresponding value in the mask array is 1
mean = [[],[],[]]
num_pixels = 0
for channel in range(3):
    values = img[:,:,channel][mask == 1] # Get the values of pixels where the mask is 1 for the current channel
    mean[channel] = np.mean(values) if len(values) > 0 else 0 # Calculate the mean of the values if there are any
    num_pixels += len(values) # Add the number of pixels with valid values to the total




#  Missing Pictures

In [ ]:
import os

# set the paths to your two folders
folder1_path = '/home/scco0002/F&E_DeepLearning_VS/F-E-Project_Part_2/dataset/dataset_large_2023_5_9_cleared/train'
folder2_path = '/home/scco0002/F&E_DeepLearning_VS/F-E-Project_Part_2/dataset/dataset_large_2023_5_9_cleared/segmentation_images'

# get the set of filenames in each folder
folder1_files = set(os.listdir(folder1_path))
folder2_files = set(os.listdir(folder2_path))

# find the set of filenames that are in folder2 but not in folder1
missing_files = folder2_files - folder1_files

# print the missing filenames
print(missing_files)


# Regeltakt

In [ ]:
import datetime
import time
starttime = datetime.datetime.now()
time.sleep(5)
endtime = datetime.datetime.now()
counter = 100

# deltatime in seconds
print(1/((endtime - starttime).seconds/counter))

# Vergleich Controller Matlab / Python

## neue Klasse für IBVS

In [ ]:
import numpy as np
from numpy.linalg import inv
from scipy import linalg

class VisualServoingController:
    def __init__(self, z=0.1):
        self.z = z

    def get_feature_jacobian(self, u, v):
        Jf = np.array([
            [-1/self.z, 0, u/self.z, u*v, -(1 + u**2), v],
            [0, -1/self.z, v/self.z, 1 + v**2, -v*u, -u]
        ], dtype=np.float64)
        return Jf

    def get_stack_feature_jacobian(self, feature_pts):
        Jfstack = None
        for i in range(feature_pts.shape[0]):
            u, v = feature_pts[i]
            Jf = self.get_feature_jacobian(u, v)
            if i == 0:
                Jfstack = Jf
            else:
                Jfstack = np.vstack((Jfstack, Jf))
        return Jfstack

    def ibvs_controller(self, Jf, f_act, f_des, kIBVS):
        e_u = f_des[:,0] - f_act[:,0]
        print(e_u)
        e_v = f_des[:,1] - f_act[:,1]
        print(e_v)
        e = np.empty((2 * len(e_u)), dtype=np.float64)
        e[::2] = e_u
        e[1::2] = e_v
        print(e)
        print(Jf.shape)
        # transform e to a column vector
        #e = e.reshape(-1, 1)
        #dp = kIBVS * (np.linalg.inv(Jf.T @ Jf)) @ Jf.T @ e

        dp = kIBVS * (np.linalg.pinv(Jf)) @ e
        return dp


In [ ]:
import numpy as np

# Actual positions of four features
f_act_good = np.array([[100, 200], [150, 250], [300, 400],[10,20]], dtype=np.float64)
# Desired positions of the four features
f_des_good = np.array([[123, 212], [142, 250], [365, 401],[84,12]], dtype=np.float64)

f_act_bad = np.array([[100, 200], [150, 250], [300, 400]],dtype=np.float64) # denken diese Werte sind linear abhängig
f_des_bad = np.array([[110, 210], [160, 260], [310, 410]],dtype=np.float64) # denken diese Werte sind linear abhängig

version = "bad"

if version == "goog":

    f_act = f_act_bad
    f_des = f_des_bad

else: # version == "good"
    f_act = f_act_good
    f_des = f_des_good

# Create a controller object
controller = VisualServoingController(z=0.1)

# Calculate the Jacobian of the features
Jf =  controller.get_stack_feature_jacobian(f_act)
# Set the gain of the IBVS controller
KiBVS = 0.01

Jf_1 = Jf.T @ Jf
#print("Jf_1",Jf_1)
#print("\n")
Jf_2 = np.linalg.inv(Jf_1)
#print(Jf_2)
#print("\n")
# Calculate the determinant
determinate = np.linalg.det(Jf_1)
#print("determinate",determinate)
#print("\n")

# Calculate the control input
dp = controller.ibvs_controller(Jf, f_act, f_des, KiBVS)
print("dp",dp)